# Spectral concentration thresholds for DPSS (multitaper) windows

Multitaper methods usually give smoother results when using a greater number of tapers, but higher order tapers suffer from poor spectral concentration (ratio of energy within versus without the bandpass, parameterized by W).

The purpose of this notebook is to find threshold levels of spectral concentration under multiple DPSS parameterizations.

In [ ]:
from ecogdata.util import dpss_windows
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, BoundaryNorm
from matplotlib.ticker import FuncFormatter
from ecoglib.vis.colormaps import nancmap

Solve DPSS windows for a number of "NW" values, and with varying window sizes.

In [ ]:
eigs_by_n = list()
# nw_range = np.arange(2.5, 10.1, 0.5)
nw_range = np.arange(1.5, 10.1, 0.5)
n_range = np.logspace(2, 4, 6).astype('i')
for n in n_range:
    max_k = int(2 * nw_range.max() + 10)
    n_eigs = np.zeros((max_k, len(nw_range)))
    for i, nw in enumerate(nw_range):
        _, eigs  = dpss_windows(int(n), nw, max_k)
        n_eigs[:, i] = np.abs(eigs)
    eigs_by_n.append(n_eigs)

In [ ]:
cm_clip = nancmap('Blues_r', overc='gray')
cm_clip = nancmap('Set2_r', overc='gray')
# diag_colors = plt.cm.autumn_r(np.linspace(0, 1, 13))
diag_colors = plt.cm.hsv_r(np.linspace(0, 1, 13))

In [ ]:
bounds = [0.8, 0.9, 0.95, 1 - 1e-2, 1 - 1e-4, 1 - 1e-6, 1 - 1e-8, 1]
b_norm = BoundaryNorm(boundaries=bounds, ncolors=256)

In [ ]:
@FuncFormatter
def one_minus(x, pos):
    return '1-{:.0e}'.format(1 - x)

### Map the eigenvalues (concentration) per taper order and per NW parameter

In [ ]:
extent = [nw_range[0] - 0.25, nw_range[-1] + 0.25, 0.5, max_k + 0.5]
f, axs = plt.subplots(2, 3, figsize=(12, 8))
for i in range(len(n_range)):
    ax = axs.ravel()[i]
    im = ax.imshow(np.abs(eigs_by_n[i]), 
                    extent=extent, 
                    norm=b_norm, 
                    clim=(0.9, 1 - (1 - max(bounds)) / 10),
                    cmap=cm_clip, origin='lower')
    ax.axis('auto')
    for k in range(0, 13, 2):
        ax.plot(nw_range, 2 * nw_range - k, lw=3, color=diag_colors[k], label='2NW - {}'.format(k))
    ax.legend(handlelength=0.5, loc='upper left', frameon=True)
    ax.set_xlim(extent[:2])
    ax.set_ylim(extent[2:])
    ax.set_xlabel('time-bandwidth product (NW)')
    ax.set_ylabel('taper order')
    ax.set_xticks(np.r_[nw_range[0], [x for x in ax.get_xticks() if extent[0] < x < extent[1]]])
    ax.set_yticks(np.r_[1, [y for y in ax.get_yticks() if extent[2] < y < extent[3]]])    
    cb = f.colorbar(im, ax=ax, extend='both', format=one_minus)
    ax.set_title('N = {}'.format(n_range[i]))
f.tight_layout()

Based on the results, the eigenvalue distributions are not much affected by sequence length. The *rough* formula appears to be that the first $2NW-m$ DPSS modes have spectral concentration greater than about $1-10^{-m}$.

An extremely high concentration (loss of 1 part per million) is achieved for $K \le 2NW - 6$. This is probably usable for NW greater than 5 or so, which could be meaningful for filter projections.

Very high concentration (1 part per 10000) is seen for $K\le 2NW-4$.

Good concentration (1 part per 100) is seen for $K\le 2NW-2$. This could even be useful at a smaller parameterization (e.g. NW=2.5).